In [1]:

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from npeet import entropy_estimators
from tensorflow_wavelets.utils.models import DWT

ModuleNotFoundError: No module named 'cv2'

In [42]:
# git clone https://github.com/gregversteeg/NPEET.git
# cd NPEET
# pip install .
# x = [[1.3],[3.7],[5.1],[2.4],[3.4]]
# y = [[1.5],[3.32],[5.3],[2.3],[3.3]]
# entropy_estimators.mi(x,y)
# Out: 0.168

In [43]:

def mutual_info_wavelet(wavelet_name, img, pd_dict:dict, shape=(240, 240)):
    # resize input image to be square
    img = cv2.resize(img, shape)
    # expand dims for the dwt transfor
    img_t = np.expand_dims(img, axis=-1)
    img_ex = np.expand_dims(img_t, axis=0)
    # concat=1 LL,LH,HL,HH in one image
    coeffs = DWT.DWT(wavelet_name, 1, "VALID", "SYMMETRIC")(tf.dtypes.cast(img_ex, tf.float32))
    # extract resault to numpy
    dtw_img = coeffs[0,:, :, 0].numpy()
    # resize the output to be like the input (maybe we need to use crop?)
    dtw_img_res = cv2.resize(dtw_img, dsize=img.shape, interpolation=cv2.INTER_LINEAR)

    flat_img = img_ex.flatten()
    flat_coeffs = dtw_img_res.flatten()
    mi = entropy_estimators.mi(flat_img, flat_coeffs)
    # print(f"Mutual Information = {mi}, for {wavelet_name} Wavelet")
    pd_dict[wavelet_name] = mi



In [ ]:

# def mutual_info_wavelet2(wavelet_name, img, pd_dict:dict, shape=(240, 240)):
    # resize input image to be square
img = cv2.imread("../src/input/Lenna_orig.png", 0)
res = dict()
shape=(240, 240)
wavelet_name = "haar"

img = cv2.resize(img, shape)
# expand dims for the dwt transfor
img_t = np.expand_dims(img, axis=-1)
img_ex = np.expand_dims(img_t, axis=0)
# concat=1 LL,LH,HL,HH in one image
coeffs = DWT.DWT(wavelet_name, concat=1)(tf.dtypes.cast(img_ex, tf.float32))


idwt = DWT.IDWT(wavelet_name, concat=1)(tf.dtypes.cast(coeffs, tf.float32))
# extract resault to numpy
dtw_img = coeffs[0,:, :, 0].numpy()
# resize the output to be like the input (maybe we need to use crop?)
dtw_img_res = cv2.resize(dtw_img, dsize=img.shape, interpolation=cv2.INTER_LINEAR)

flat_img = img_ex.flatten()
flat_coeffs = dtw_img_res.flatten()
mi = entropy_estimators.mi(flat_img, flat_coeffs)
# print(f"Mutual Information = {mi}, for {wavelet_name} Wavelet")
pd_dict[wavelet_name] = mi



In [44]:
img = cv2.imread("../src/input/Lenna_orig.png", 0)
res = dict()
mutual_info_wavelet("haar", img, res)
mutual_info_wavelet("db2", img, res)
mutual_info_wavelet("db3", img, res)
mutual_info_wavelet("coif2", img, res)
mutual_info_wavelet("coif3", img, res)
mutual_info_wavelet("sym2", img, res)
mutual_info_wavelet("sym3", img, res)
mutual_info_wavelet("bior1.3", img, res)
mutual_info_wavelet("bior2.2", img, res)
mutual_info_wavelet("rbio1.3", img, res)
mutual_info_wavelet("rbio2.2", img, res)


In [45]:
df = pd.DataFrame(list(res.items()), columns=['wavelet_name', 'mutual_information'])
df

,wavelet_name,mutual_information
0,haar,0.300046
1,db2,0.248661
2,db3,0.240524
3,coif2,0.239007
4,coif3,0.244888
5,sym2,0.247446
6,sym3,0.241551
7,bior1.3,0.247472
8,bior2.2,0.247495
9,rbio1.3,0.252135
